In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

C:\Users\nicoesse\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load synthetic queries
syn_queries = '../02_nps_api_data/synthetic_queries.csv'
synthetic_queries_df = pd.read_csv(syn_queries)

In [3]:
# Train/validation split
train_df, val_df = train_test_split(synthetic_queries_df, test_size=0.2, random_state=42)

# Reduce the dataset size for quick testing
train_df = train_df.sample(n=100, random_state=42)
val_df = val_df.sample(n=20, random_state=42)

print(len(train_df), len(val_df))



100 20


In [4]:
# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [5]:
# Initialize tokenizer and model
model_name = 'google/flan-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Preprocess data

target = 'api_call.endpoint'

def preprocess_function(examples):
    inputs = [f"{target}: {query}" for query in examples['query']]
    model_inputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    labels = tokenizer(examples[target], max_length=512, padding='max_length', truncation=True).input_ids
    model_inputs['labels'] = labels
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map: 100%|██████████| 20/20 [00:00<00:00, 2000.10 examples/s]


In [8]:
# Define training arguments
#training_args = TrainingArguments(
#    output_dir='./results',
#    eval_strategy='epoch',
#    learning_rate=2e-5,
#    per_device_train_batch_size=8,
#    per_device_eval_batch_size=8,
#    num_train_epochs=1, #3
#    weight_decay=0.01,
#    logging_dir='./logs',
#)

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='steps',
    eval_steps=1000,  # Evaluate every 1000 steps
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reduced number of epochs for quick testing
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train and evaluate
trainer.train()
trainer.evaluate()



C:\Users\nicoesse\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Save the model
model.save_pretrained('./nps_model_endpoint')
tokenizer.save_pretrained('./nps_model_endpoint')